In [38]:
# Load Imports
import numpy as np
import nibabel as nib
from data_loader import *
import matplotlib.pyplot as plt


import os
import re
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
from metrics import dice_coef


%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [11]:
# Aux. Functions
def loadVolume(patient_name, slice_type):
    ''' Load volume created by predicting partient side'''
    volumes_fp = '../3D_test/Prob_Vols/'
    vol_side_fn = (volumes_fp +
                   patient_name +
                   '_mask_pred_' +
                   slice_type +
                   '.nii')
    return (nib.load(vol_side_fn)).get_data()


def compute3DMask(side_vol, top_vol, back_vol, thresh=None):
    prob_vol_arr = np.array([vol_side, vol_top, vol_back])
    avg_vol = np.mean(prob_vol_arr, axis=0)
    if thresh:
        thresh_vol = thresholdMask(avg_vol, threshold=thresh)
        return thresh_vol
    else:
        return avg_vol

def show_slices(slices):
    """ Function to display row of image slices """
    fig, axes = plt.subplots(1, len(slices))
    for i, slice in enumerate(slices):
      axes[i].imshow(slice.T, cmap="gray", origin="lower")

def show_slice(slice):
    plt.imshow(slice.T, cmap="gray", origin="lower")
    
def normalizeImg(x):
    # Normalize x
    max_val = np.max(x)
    min_val = np.min(x)
    norm_x = (x-min_val)/(max_val-min_val + 1e-7)
    return norm_x

def thresholdMask(mask, threshold=0.56):
    mask[mask >= threshold] = 1.0
    mask[mask < threshold] = 0.0
    return mask
    


In [3]:
# Load Patient Input Volume
patient_input = '../test_set_mri_new/test_set_mri/IXI101-Guys-0749-T1.nii'
defaced_patient_input = '../test_set_mri_new/test_set_mri/IXI101-Guys-0749-T1_defaced.nii'
normal = nib.load(patient_input).get_data()
defaced = nib.load(defaced_patient_input).get_data()


In [4]:
# Load probabilistic volumes
patient_name = 'IXI101-Guys-0749-T1'
# Load side volume
slice_type = 'side'
vol_side = loadVolume(patient_name, slice_type)
# Load top volume
slice_type = 'top'
vol_top = loadVolume(patient_name, slice_type)
# Load back volume
slice_type = 'back'
vol_back = loadVolume(patient_name, slice_type)


In [14]:
''' Compute the average of each element in matrix across multiple matrices '''
avg_vol = compute3DMask(vol_side,
                           vol_top,
                           vol_back,
                           thresh=None)


In [39]:
''' Compute the dice scores of the 3D Masks '''
dice_stats = []
input_dir = '../3D_test'
total_files = get_file_list_from_dir(input_dir, y_label='_mask')
for norm_input, mask_input in total_files:
    # Patient name
    patient_name = re.search('\.\./3D_test/(.*)\.nii',
                             norm_input).group(1)
    print(patient_name)
    # Load mask
    mask_3d = nib.load(mask_input).get_data()

    # Load side volume
    slice_type = 'side'
    vol_side = loadVolume(patient_name, slice_type)
    # Load top volume
    slice_type = 'top'
    vol_top = loadVolume(patient_name, slice_type)
    # Load back volume
    slice_type = 'back'
    vol_back = loadVolume(patient_name, slice_type)
    # Compute the avg_vol
    avg_vol = compute3DMask(vol_side,
                           vol_top,
                           vol_back,
                           thresh=None)
    # Compute the dice score b.w. the 3 volumes
    patient_ds = dice_coef(mask_3d, 
                           avg_vol,
                           threshold = 0.59)
    print('Dice Coefficient : ', patient_ds)
    dice_stats.append(patient_ds)
    


updated
IXI336-Guys-0904-T1
Dice Coefficient :  0.07710767258803575
IXI228-Guys-0822-T1
Dice Coefficient :  0.8137100662949605
IXI455-Guys-0981-T1
Dice Coefficient :  0.8307710798909173
IXI428-Guys-0996-T1
Dice Coefficient :  0.8021003993837591
IXI108-Guys-0865-T1
Dice Coefficient :  0.8988576461633322
IXI566-HH-2535-T1
Dice Coefficient :  0.8417337014406637
IXI510-IOP-1143-T1
Dice Coefficient :  0.8181631370983367
IXI543-IOP-1148-T1
Dice Coefficient :  0.8053056604589857
IXI212-HH-1643-T1
Dice Coefficient :  0.8805963621945009
IXI170-Guys-0843-T1
Dice Coefficient :  0.830613952327826
IXI055-Guys-0730-T1
Dice Coefficient :  0.8482017074555555
IXI167-HH-1569-T1
Dice Coefficient :  0.892361336829256
IXI174-HH-1571-T1
Dice Coefficient :  0.8444770559143729
IXI559-HH-2394-T1
Dice Coefficient :  0.7831231218107441
IXI028-Guys-1038-T1
Dice Coefficient :  0.8517386223919338
IXI267-HH-1772-T1
Dice Coefficient :  0.9098852308546139
IXI092-HH-1436-T1
Dice Coefficient :  0.9176675285759165
IXI446

In [43]:
# Display the statistics 
from scipy import stats
print(stats.describe(dice_stats))
print('Mean dice:', np.mean(dice_stats))

DescribeResult(nobs=50, minmax=(0.07710767258803575, 0.9328458308233589), mean=0.8143336038548036, variance=0.015279195244804068, skewness=-4.4441020637084, kurtosis=23.707513585960996)
Mean dice: 0.8143336038548036
